In [1]:
using DataFrames, TableOperations, Tables, Random
using Recommender: Movielens100k, load_dataset, ratio_split, ItemkNN, evaluate_u2i, PrecisionAtK, RecallAtK, NDCG

In [2]:
ml100k = Movielens100k()
download(ml100k)
rating, user, movie = load_dataset(ml100k);

In [3]:
rating = rating |> TableOperations.filter(x->Tables.getcolumn(x, :rating) >= 4)

TableOperations.Filter{var"#1#2", CSV.File{false}}(var"#1#2"(), CSV.File("/Users/keisuke.yanagi/workspace/Recommender.jl/src/dataset/../../dataset/movielens100k/u.data"):
Size: 100000 x 4
Tables.Schema:
 :userid     Int64
 :movieid    Int64
 :rating     Int64
 :timestamp  Int64)

In [4]:
Random.seed!(1234);
train_valid_table, test_table = ratio_split(rating, 0.8)

train_table, valid_table = ratio_split(train_valid_table, 0.8)
length(Tables.rows(train_table)), length(Tables.rows(valid_table)), length(Tables.rows(test_table))


(35440, 8860, 11075)

In [5]:
prec10 = PrecisionAtK(10)
recall10 = RecallAtK(10)
ndcg10 = NDCG(10)
metrics = [prec10, recall10, ndcg10]

3-element Vector{Recommender.AbstractMetric}:
 PrecisionAtK(10)
 RecallAtK(10)
 NDCG(10)

In [6]:
using TreeParzen

In [7]:
space = Dict(
    :topk=>HP.QuantUniform(:topk, 10., 500., 1.),
    :shrink=>HP.LogUniform(:shrink, log(1e-3), log(1e3)),
    :weighting=>HP.Choice(:weighting, 
        [
            Dict(:weighting=>:dummy, :weighting_at_inference=>false),
            Dict(:weighting=>:tfidf, :weighting_at_inference=>false),
            Dict(:weighting=>:bm25, :weighting_at_inference=>HP.Choice(:weighting_at_inference, [true, false]))
        ]
    ),
    :normalize=>HP.Choice(:normalize, [true, false])
)

Dict{Symbol, TreeParzen.Types.AbstractDelayed} with 4 entries:
  :weighting => Choice(Param(:weighting, RandIndex(3)), Dict{Symbol, Any}[Dict(…
  :topk      => QuantUniform(:topk, QuantUniform(10.0, 500.0, 1.0))
  :normalize => Choice(Param(:normalize, RandIndex(2)), Bool[1, 0])
  :shrink    => LogUniform(:shrink, LogUniform(-6.90776, 6.90776))

In [8]:
function invert_output(params)
    k = convert(Int, params[:topk])
    model = ItemkNN(k, params[:shrink],params[:weighting][:weighting],params[:weighting][:weighting_at_inference],params[:normalize])
    result = evaluate_u2i(model, train_table, valid_table, metrics, 10, col_user=:userid, col_item=:movieid, col_rating=:rating, drop_history=false)
    @show params, result
    return -result[end]
end

invert_output (generic function with 1 method)

In [9]:
best = fmin(invert_output, space, 20, logging_interval=-1)

(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :bm25, :weighting_at_inference => false), :topk => 326.0, :normalize => false, :shrink => 4.881008704229105), (NDCG(10) = 0.09069945040388037, RecallAtK(10) = 0.10554575881777516, PrecisionAtK(10) = 0.06177777777777812))
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :dummy, :weighting_at_inference => false), :topk => 482.0, :normalize => true, :shrink => 693.233059620366), (NDCG(10) = 0.1190386241842776, RecallAtK(10) = 0.12041231489019875, PrecisionAtK(10) = 0.0784444444444447))
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :bm25, :weighting_at_inference => true), :topk => 210.0, :normalize => true, :shrink => 0.027243650448104388), (NDCG(10) = 0.06544378188164292, RecallAtK(10) = 0.08545396776483244, PrecisionAtK(10) = 0.04411111111111133))
(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :du

┌ Info: fmin: 20 / 20 trials carried out
└ @ TreeParzen.API /Users/keisuke.yanagi/.julia/packages/TreeParzen/Iw2mh/src/API.jl:176
┌ Info: Successfully completed fmin 
└ @ TreeParzen.API /Users/keisuke.yanagi/.julia/packages/TreeParzen/Iw2mh/src/API.jl:231


Dict{Symbol, Any} with 4 entries:
  :weighting => Dict{Symbol, Any}(:weighting=>:dummy, :weighting_at_inference=>…
  :topk      => 482.0
  :normalize => true
  :shrink    => 693.233

In [10]:
best_model = ItemkNN(convert(Int, best[:topk]), best[:shrink],best[:weighting][:weighting],best[:weighting][:weighting_at_inference],best[:normalize])
evaluate_u2i(best_model, train_valid_table, test_table, metrics, 10, col_user=:userid, col_item=:movieid, col_rating=:rating, drop_hisotory=false)

(NDCG(10) = 0.1588432604730927,
 RecallAtK(10) = 0.1416637543183754,
 PrecisionAtK(10) = 0.10391304347826046,)